In [41]:
import torch
from torch.utils.data import Dataset,random_split,DataLoader
from torch.nn import Linear,ReLU,Sigmoid,Tanh,Softmax,Module,BCELoss
from torch.optim import Adam,SGD
from torch.nn.init import xavier_uniform_,kaiming_uniform_
import pandas as pd
from torch import Tensor
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score

In [33]:
class fetchData(Dataset):
    def __init__(self,path):
        data=pd.read_csv(path,header=None,)
        self.x=data.values[:,:-1]
        self.x=self.x.astype('float32')
        self.y=data.values[:,-1]
        self.y=LabelEncoder().fit_transform(self.y)
        self.y=self.y.astype('float32')
        self.y=self.y.reshape(len(self.y),1)

        
    def __len__(self):
        return len(self.y)
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    def split(self,ratio=0.3):
        size1=round(ratio*len(self.x))
        size2=len(self.x) -size1
        return random_split(self,[size1,size2])
def getdata(path,ratio=0.5):
    obj=fetchData(path)
    train,test=obj.split(ratio)
    train=DataLoader(train,shuffle=True,batch_size=30)
    test=DataLoader(test,shuffle=False,batch_size=1000)
    return train,test

In [34]:
class NeuralNetwork(Module):
    
    def __init__(self,input_size):
        super(NeuralNetwork,self).__init__()
        self.layer1=Linear(input_size,10)
        kaiming_uniform_(self.layer1.weight,nonlinearity='relu')
        self.activation1=ReLU()
        
        self.layer2=Linear(10,6)
        kaiming_uniform_(self.layer2.weight,nonlinearity='relu')
        self.activation2=ReLU()
        
        self.layer3=Linear(6,1)
        xavier_uniform_(self.layer3.weight)
        self.activation3=Sigmoid()
    def forward(self,data):
        data=self.layer1(data)
        data=self.activation1(data)
        data=self.layer2(data)
        data=self.activation2(data)
        data=self.layer3(data)
        data=self.activation3(data)
        return data
def training(train_data,Model):
    loss_func=BCELoss()
    optimizer=SGD(Model.parameters(),lr=0.01,momentum=0.85)
    for(x,y) in train_data:
        optimizer.zero_grad()
        result=Model(x)
        loss=loss_func(result,y)
        loss.backward()
        optimizer.step()
model=NeuralNetwork(5)
model(Tensor([1,2,3,4,5]))

tensor([0.0009], grad_fn=<SigmoidBackward>)

In [35]:
path='ionosphere.csv'
train,test=getdata(path,0.3)
input_size=len(train.dataset[0][0])
model=NeuralNetwork(input_size)
training(train,model)

In [60]:
def evalute(test_data,model):
    acc_result=[]
    acc_y=[]
    for(x,y) in test_data:
        result=model(x)
        result=result.detach().numpy()
        y=y.detach().round()
        result=result.round()
        acc_result.append(result)
        acc_y.append(y)
        acc_result=np.vstack(acc_result)
        print(round(accuracy_score(acc_result,y),2)*100,'%')


In [100]:
evalute(test,model)

59.0 %
